In [2]:
%load_ext autoreload

# Auto reloading causes the kernel to reload the libraries we have
%autoreload 2

import requests


# Functions

In [3]:
# Low-level API calls

def get_current_stats(base='http://localhost:8089/'):
    # Get the current status
    resp = requests.get(base + 'stats/requests')
    return resp.json()

def set_target_user_count(locust_count, hatch_rate=50, base='http://localhost:8089/'):
    # set the target state
    resp = requests.post(base+'swarm', data={'locust_count':locust_count, 'hatch_rate': hatch_rate})
    try:
        resp_json = resp.json()
        return resp_json['success']
    except:
        return False
    
def stop_test(base='http://localhost:8089/'):
    # Stop the test
    resp = requests.get(base+'stop')
    try:
        resp_json = resp.json()
        return resp_json['success']
    except:
        return False
    
def reset_stats(base='http://localhost:8089/'):
    # reset stats
    resp = requests.get(base+'stats/reset')
    if resp.text == 'ok':
        return True
    else:
        return False

In [41]:
import threading
import numpy as np

import time
class TimerClass:
    def __init__(self):
        self.start_time = time.time()

    def tic(self):
        self.start_time = time.time()

    def toc(self):
        elapsed = time.time() - self.start_time
        return elapsed

    def toc_print(self):
        elapsed = time.time() - self.start_time
        print('{:4.02f}'.format(elapsed))
        return elapsed

class WorkerThread(threading.Thread):
    def __init__(self, parent, sleep_time=2):
        super(WorkerThread, self).__init__()
        # if daemon is true this thread will die when the main thread dies
        self.daemon = True
        self.stop_signal = False
        self.sleep_time = sleep_time
        self.loop_timer = TimerClass()
        self.parent = parent
        
    def run(self):
        while not self.stop_signal:
            self.loop_timer.tic()
            try:
                stats = self.parent.get_stats()
                self.parent.reset_remote_stats()
                stats['time'] = time.time()
                self.parent.temp_stats.append(stats)
                if len(self.parent.temp_stats) > self.parent.temp_stat_max_len:
                    del self.parent.temp_stats[0]
            except Exception as e:
                print('Got Exception: ' + str(e))
            elapsed = self.loop_timer.toc()
            time.sleep(self.sleep_time - elapsed)

class DdslLoadTester:
    def __init__(self, base='http://localhost:8089/',hatch_rate=50,temp_stat_max_len=5):
        super(DdslLoadTester, self).__init__()
        self.base = base
        self.hatch_rate = hatch_rate
        self.worker_thread = None
        self.temp_stats = []
        self.temp_stat_max_len = temp_stat_max_len
        
    def get_state(self):
        resp = get_current_stats(self.base)
        return resp['state']
    
    def get_stats(self):
        return get_current_stats(self.base)
    
    def reset_remote_stats(self):
        return reset_stats(self.base)
        
    def reset_temp_stats(self):
        self.temp_stats = []
        return True
    
    def get_temp_stats(self):
        tmp_stats = self.temp_stats
        self.reset_temp_stats()
        return tmp_stats
    
    def set_count(self, new_count):
        return self.change_count(new_count)
    
    def change_count(self, new_count):
        return set_target_user_count(new_count, self.hatch_rate, self.base)
    
    def stop_test(self):
        return stop_test(self.base)
    
    def start_capturing(self):
        curr_state = self.get_state()
        if curr_state != 'running':
            raise Exception('You should start the test first by calling tester.set_count(). state:' + curr_state)
        
        self.worker_thread = WorkerThread(self)
        self.worker_thread.start()
        
    def stop_capturing(self):
        if self.worker_thread is not None:
            self.worker_thread.stop_signal = True
            return True
        else:
            return True
        
def get_stats_arr(stats, key):
    return [stats[i][key] for i in range(len(stats))]

# Testing Functions

In [5]:
get_current_stats()

{'current_response_time_percentile_50': 27.753114700317383,
 'current_response_time_percentile_95': 110,
 'errors': [],
 'fail_ratio': 0.0,
 'state': 'running',
 'stats': [{'avg_content_length': 4.0,
   'avg_response_time': 42.931657776868256,
   'current_rps': 6.8,
   'max_response_time': 138.3364200592041,
   'median_response_time': 26.89194679260254,
   'method': 'GET',
   'min_response_time': 22.538423538208008,
   'name': '/function/pyfibo/',
   'num_failures': 0,
   'num_requests': 802},
  {'avg_content_length': 4.0,
   'avg_response_time': 42.931657776868256,
   'current_rps': 6.8,
   'max_response_time': 138.3364200592041,
   'median_response_time': 26.89194679260254,
   'method': None,
   'min_response_time': 22.538423538208008,
   'name': 'Total',
   'num_failures': 0,
   'num_requests': 802}],
 'total_rps': 6.8,
 'user_count': 50}

In [7]:
set_target_user_count(50)

True

In [16]:
reset_stats()

True

In [29]:
stop_test()

True

In [17]:
get_current_stats()

{'current_response_time_percentile_50': 25.371313095092773,
 'current_response_time_percentile_95': 89.7066593170166,
 'errors': [],
 'fail_ratio': 0.0,
 'state': 'running',
 'stats': [{'avg_content_length': 4.0,
   'avg_response_time': 36.92316157477243,
   'current_rps': 2.0,
   'max_response_time': 105.74507713317871,
   'median_response_time': 25.371313095092773,
   'method': 'GET',
   'min_response_time': 23.4682559967041,
   'name': '/function/pyfibo/',
   'num_failures': 0,
   'num_requests': 28},
  {'avg_content_length': 4.0,
   'avg_response_time': 36.92316157477243,
   'current_rps': 2.0,
   'max_response_time': 105.74507713317871,
   'median_response_time': 25.371313095092773,
   'method': None,
   'min_response_time': 23.4682559967041,
   'name': 'Total',
   'num_failures': 0,
   'num_requests': 28}],
 'total_rps': 2.0,
 'user_count': 50}

# Testing Object

In [42]:
load_tester = DdslLoadTester(hatch_rate=100)
load_tester.change_count(50)

True

In [43]:
load_tester.start_capturing()

In [46]:
temp_stats = load_tester.get_temp_stats()

In [47]:
get_stats_arr(temp_stats, 'current_response_time_percentile_50')

[29.175281524658203,
 29.175281524658203,
 40.78841209411621,
 40.78841209411621,
 33.73289108276367]

In [38]:
load_tester.stop_capturing()

True

In [27]:
load_tester.stop_test()

True